# Playing with PyPI (data)

Many thanks to Seth M. Larson for making these available: https://github.com/sethmlarson/pypi-data

## Download data

Note that `DuckDB` supports reading directly from remote parquet files (and performs column-pruning and other optimizations to reduce network load) but we'll be executing several queries against these data and so we're better off downloading the files to begin with.

In [ ]:
from pathlib import Path

filenames = [
    "deps.parquet",
    "maintainers.parquet",
    "package_urls.parquet",
    "packages.parquet",
    "scorecard_checks.parquet",
    "wheels.parquet",
]

folder = Path("pypi")
folder.mkdir(exist_ok=True)

for filename in filenames:
    path = folder / filename
    if not path.exists():
        import urllib.request

        urllib.request.urlretrieve(
            f"https://storage.googleapis.com/ibis-tutorial-data/pypi/{filename}",
            path,
        )

In [ ]:
import ibis
from ibis import _

ibis.options.interactive = True

In [ ]:
con = ibis.duckdb.connect()

In [ ]:
for filename in filenames:
    path = folder / filename
    con.read_parquet(path, table_name=filename.split(".")[0])

In [ ]:
con.list_tables()

In [ ]:
deps = con.tables.deps

deps

In [ ]:
maintainers = con.tables.maintainers

maintainers

In [ ]:
packages = con.tables.packages

packages

## Using `.sql` as an escape hatch

Ibis is pretty good about abstracting away the underlying execution engine, but sometimes you _want_ to talk directly to the engine.

Maybe you:
- have an existing SQL query from a data engineer that you want to build on
- need to access a backend-specific function that isn't exposed in Ibis
- really want to write some SQL

If you do, then you can call `.sql` on a connection (or an existing expression!) and write out your SQL query.

(You can even pass a `dialect` flag if you are more comfortable writing SQL in a dialect different than the one your engine expects!)

### Looking for typo squatters

DuckDB has a `jaccard` function for doing Jaccard similarity checks on strings.  But there's no `jaccard` method in `ibis` -- what to do?  

We'll write a short SQL query:

In [ ]:
# packages.name.jaccard("pandas")  # <--this is not a thing (yet!)

In [ ]:
expr = packages.sql(
    "SELECT name, version, jaccard(packages.name, 'pandas') as sim from packages",
    dialect="duckdb",
)  # the dialect keyword is redundant here

expr

We should probably sort that `sim` column to see which packages have higher similarity.

Even though we called out to `sql` for the Jaccard function call, we don't need to stay in SQL-land moving forward, just chain as usual!

In [ ]:
expr.order_by(_.sim.desc())

### Exercise 1

Try swapping out `jaccard` for a different similarity function available in DuckDB (https://duckdb.org/docs/sql/functions/char#text-similarity-functions) -- or you can try looking for similar PyPI usernames using the `maintainers` table!

### Solution

One possible solution:

In [ ]:
%load solutions/nb03-ex01-jaro.py

In [ ]:
%load solutions/nb03-ex01-maint.py

## Integration with other tools

Plotting data is quite helpful!  Recent support for the `__dataframe__` protocol has made it easier than ever to use Ibis with various plotting libraries.

### Histogram of maintainer count

We'll start by computing the number of packages that have `n` maintainers, as defined by having write permissions on PyPI.

In [ ]:
maintainer_counts = (
    maintainers.group_by("package_name")
    .agg(maintainers=_.count())
    .group_by("maintainers")
    .agg(count=_.count())
    .order_by(_.maintainers)
)

maintainer_counts

We'll plot a histogram of the maintainer counts using `altair`.

In [ ]:
import altair as alt

alt.__version__

In [ ]:
ibis.__version__

In [ ]:
chart = (
    alt.Chart(maintainer_counts)
    .mark_bar()
    .encode(x="maintainers", y=alt.Y("count", scale=alt.Scale(type="log")))
)

chart

We can also use `plotnine`!

In [ ]:
from plotnine import aes, geom_histogram, ggplot, scale_y_log10, geom_bar

In [ ]:
(
    ggplot(maintainer_counts, aes(x="maintainers", y="count"))
    + geom_bar(stat="identity")
    + scale_y_log10()
)

Finally, since we can always call `to_pandas` or `to_pyarrow` -- it's easy to connect an Ibis result to your favorite PyData tools.

### Exercise 2

What's the deal with that spike at 12 maintainers?

**Note**: if you would like to return more rows in interactive mode, you can set the row-count e.g. `ibis.options.repr.interactive.max_rows = 20`

### Solution

In [ ]:
%load solutions/nb03-ex02-counts.py

In [ ]:
%load solutions/nb03-ex02-ftw.py

In [ ]:
%load solutions/nb03-ex02-ftw-maintainers.py

## UDFs

Modern SQL engines have a _lot_ of features, but there's always going to be _something_ you want to do that isn't built-in.  Maybe you have a text classifier you want to run over a dataset?

We'll tackle something a little less ML-related, and figure out if we need to update our PyPI data pull.

In [ ]:
pp = (
    packages.order_by(_.downloads.desc())
    .select("name", "version", "downloads")
    .limit(10)
)

In [ ]:
pp

In [ ]:
from ibis.expr.operations import udf

In [ ]:
import json

import requests


@udf.scalar.python
def check_version(name: str, version: str) -> bool:
    r = requests.get(f"https://pypi.org/pypi/{name}/json")
    return (
        version == sorted(list(json.loads(r.content.decode())["releases"].keys()))[-1]
    )

In [ ]:
pp.mutate(is_latest=check_version(pp.name, pp.version))

### Exercise 3

Above we used a UDF to look up whether the version of a given package in our dataset was the latest release on PyPI. 

Let's add another UDF to check whether any of the packages in our top-10 downloads have known vulnerabilities.

The API endpoint for checking vulnerabilities is `"https://pypi.org/pypi/{name}/{version}/json"` -- if you load in the JSON from that `GET`, there is a `vulnerabilities` key that will be a list of known vulnerabilities.

Write a UDF that grabs this information, and if it is present, pull out the `aliases` and `fixed_in` subfields.

Here is a sample API response for a known vulnerability:

```
GET /pypi/Django/3.0.2/json HTTP/1.1
Host: pypi.org
Accept: application/json

{
    "info": {},
    "last_serial": 12089094,
    "releases": {},
    "urls": [],
    "vulnerabilities": [
        {
            "aliases": [
                "CVE-2021-3281"
            ],
            "details": "In Django 2.2 before 2.2.18, 3.0 before 3.0.12, and 3.1 before 3.1.6, the django.utils.archive.extract method (used by \"startapp --template\" and \"startproject --template\") allows directory traversal via an archive with absolute paths or relative paths with dot segments.",
            "summary": "A shorter summary of the vulnerability",
            "fixed_in": [
                "2.2.18",
                "3.0.12",
                "3.1.6"
            ],
            "id": "PYSEC-2021-9",
            "link": "https://osv.dev/vulnerability/PYSEC-2021-9",
            "source": "osv",
            "withdrawn": null
        },
    ]
}
```

### Solution

In [ ]:
%load solutions/nb03-ex03.py

## More exercises / questions:

If you've finished all the exercises and still have some time or want to explore using Ibis further, here are some more questions to try to answer using the PyPI data.

### Exercise 4: Which maintainers have the most downloads?

Which package maintainers on PyPI have the most downloads across all the packages they maintain?

### Solution

In [ ]:
%load solutions/nb03-ex04.py

### Exercise 5: What packages depend on `ibis-framework`?

Or fill in your package name of choice

### Solution

In [ ]:
%load solutions/nb03-ex05.py

### Exercise 6: What packages depend on things I maintain?

Do you publish any PyPI packages?  Try looking up which packages depend on those packages.
Or you can try looking up some of the Ibis maintainers dependency graphs (`gforsyth`, `jcrist`, `Charles.Cloud`)

### Solution

In [ ]:
%load solutions/nb03-ex06.py

### Exercise 7: What are the top pytest extensions?

We love `pytest` and so do a bunch of other people.  Their plugins/extensions also follow a naming convention that they start with `pytest-`, so try to find out which `pytest` extensions are the most depended on.

### Solution

In [ ]:
%load solutions/nb03-ex07.py

### Exercise 8: Find the top 20 most depended on packages that have only one maintainer

Sometimes people refer to bus factors, other times we think of someone winning the lottery and moving to a remote cabin in the woods (this is my plan). Which packages would be left without an active maintainer if one person is no longer available to publish?

### Solution

In [ ]:
%load solutions/nb03-ex08.py